# Jupyter-Scala

This jupyter-scala kernel is installed following the (non complete yet) documentation in the [official page ](https://github.com/alexarchambault/jupyter-scala/tree/e1e3db4b9f013a2d8b785001ae6a390897bd20fa#classpathadd)

This small notebook example shows the following examples:

1. A simple scala example
2. Importing from maven repositories
3. importing libraries into the project
4. Working with Spark
5. Connecting with Spark to Cassandra

In [1]:
println("test")
println("this is a talk test")

test
this is a talk test


In [2]:
def compare(a:Int, b:Int):Boolean = {
    a.equals(b)
}

defined function compare

In [3]:
def foo(a: String, b: String): String ={
    println(a + b)
    a.+(b)
}

defined function foo

In [4]:
compare(3,4)

res3: Boolean = false

In [5]:
compare(4,4)

res4: Boolean = true

In [6]:
foo("string1 ", "String 2")

string1 String 2


res5: String = "string1 String 2"

In [8]:
show(1)
show(compare(3,3))

1

true

### Adding paths to libraries

Sometimes we need to add resources to ivy that are not already in the default configuration

In [9]:
//classpath.addRepository("http://repository.cloudera.com/content/repositories/releases")

In [10]:
classpath.addRepository("http://dl.bintray.com/scalaz/releases")
classpath.addRepository("https://repo1.maven.org/maven2/")

### loading libraries from the repository directly

Also there are many libraries that we'll need to be able to work with spark

In [6]:
classpath.add("org.apache.spark" % "spark-core_2.11" % "1.6.0" )

Adding 130 artifact(s)


The following is the same as before, the %% adds the scala version at the end and generates the corerct path

In [1]:
classpath.add("org.apache.spark" %% "spark-core" % "1.6.0")

Adding 130 artifact(s)


In [12]:
classpath.add("org.apache.spark" %% "spark-sql" % "1.6.0")

Adding 11 artifact(s)


There are some issues with Cassandra Spark Connector versions >1.4 where it has a problem with the guava version

In [4]:
//classpath.addRepository("http://spark-packages.org/package/datastax/spark-cassandra-connector")
//classpath.add("com.datastax.spark" %% "spark-cassandra-connector" % "1.6.0-M1")

Adding 11 artifact(s)


In [9]:
//classpath.addRepository("http://spark-packages.org/package/datastax/spark-cassandra-connector")
//classpath.add("com.datastax.spark" %% "spark-cassandra-connector" % "1.5.0")

Adding 11 artifact(s)


In [13]:
//classpath.addRepository("http://spark-packages.org/package/datastax/spark-cassandra-connector")
classpath.add("com.datastax.spark" %% "spark-cassandra-connector" % "1.3.0")

Adding 7 artifact(s)


In [14]:
classpath.add("org.apache.cassandra" % "cassandra-all" % "2.2.5")
//classpath.add("org.apache.cassandra" %% "cassandra-all" % "2.2.5")

Adding 31 artifact(s)


In [16]:
import org.apache.spark.{ SparkConf, SparkContext }
import org.apache.spark.sql.SQLContext
import com.datastax.spark.connector._

import java.text.SimpleDateFormat
import java.util.Calendar
import java.util.UUID
import java.io.File
import java.io.FileWriter
import scala.io.Source
import org.apache.log4j.PropertyConfigurator

import org.apache.spark.{ SparkConf, SparkContext }
import org.apache.spark.sql.SQLContext
import com.datastax.spark.connector._
import java.text.SimpleDateFormat
import java.util.Calendar
import java.util.UUID
import java.io.File
import java.io.FileWriter
import scala.io.Source
import org.apache.log4j.PropertyConfigurator

Starting Spark configuration and process

In [18]:
//this is an example spark configuration, the setMaster is absolutely needed to be able to connect to the spark service
val sparkConf = new SparkConf()
      .setAppName("JupyterScalaTest")
      .setMaster("local")

sparkConf: SparkConf = org.apache.spark.SparkConf@61106360

In [19]:
val sc = new SparkContext(sparkConf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/leo/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.10/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/leo/.coursier/cache/v1/https/repo1.maven.org/maven2/ch/qos/logback/logback-classic/1.1.3/logback-classic-1.1.3.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
16/03/01 18:51:55 INFO SparkContext: Running Spark version 1.6.0
16/03/01 18:51:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/03/01 18:51:56 WARN Utils: Your hostname, deepl resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on i

sc: SparkContext = org.apache.spark.SparkContext@3a3bfcee

Now we have a Spark Context instance with which we can play ;)

#### Playing a bit with Spark

In [20]:
val textFile = sc.textFile("/home/leo/installers/spark-1.6.0-bin-hadoop2.6/README.md")

textFile: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1] at textFile at Main.scala:32

In [21]:
textFile.count()

res20: Long = 95L

In [22]:
textFile.first()

res21: String = "# Apache Spark"

In [23]:
val linesWithSpark = textFile.filter(line => line.contains("Spark"))

linesWithSpark: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at Main.scala:32

In [25]:
val sparkTextCountLines = linesWithSpark.count()

sparkTextCountLines: Long = 17L

In [26]:
println(sparkTextCountLines )

17


In [27]:
sc.stop()

### Now Playing with Cassandra

The examples are based on the official documentation of Datastax Spark Cassandra Connector taken from 
[here](https://github.com/datastax/spark-cassandra-connector/blob/master/doc/0_quick_start.md)

In [28]:
//Now we create a configuration with the cassandra elements
val sparkCassaConf = new SparkConf()
      .setAppName("JupyterScalaTest-Cassandra")
      .setMaster("local")
      .set("spark.cassandra.connection.host", "127.0.0.1")
      //.set("spark.cassandra.connection.port", "")
      //.set("spark.cassandra.auth.username", "user")
      //.set("spark.cassandra.auth.password", "pwd_cassandra")

val scc = new SparkContext(sparkCassaConf)

sparkCassaConf: SparkConf = org.apache.spark.SparkConf@220d48e4
scc: SparkContext = org.apache.spark.SparkContext@2ffa5098

In [29]:
val sqlContext = new SQLContext(scc)

sqlContext: SQLContext = org.apache.spark.sql.SQLContext@a41f055

In [30]:
val rdd = scc.cassandraTable("test", "kv")

rdd: rdd.CassandraTableScanRDD[CassandraRow] = CassandraTableScanRDD[0] at RDD at CassandraRDD.scala:18

In [31]:
println(rdd)

CassandraTableScanRDD[0] at RDD at CassandraRDD.scala:18


In [32]:
rdd.columnNames 

res31: ColumnSelector = AllColumns

In [33]:
rdd.isEmpty

res32: Boolean = false

In [34]:
rdd.count()

res33: Long = 6L

In [35]:
println(rdd.count)
println(rdd.first)
println(rdd.map(_.getInt("value")).sum) 

6
CassandraRow{key: key5, value: 5}
17.0


In [36]:
val collection = scc.parallelize(Seq(("key10", 3), ("key11", 4)))
collection.saveToCassandra("test", "kv", SomeColumns("key", "value")) 

collection: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[5] at parallelize at Main.scala:33

In [37]:
rdd.count()

res36: Long = 8L

In [38]:
println(rdd.count)
println(rdd.first)
println(rdd.map(_.getInt("value")).sum) 

8
CassandraRow{key: key5, value: 5}
24.0
